In [1]:
import pandas as pd
import lightgbm as lbm
from sklearn.model_selection import RepeatedKFold,train_test_split,cross_val_score
import os
import glob


In [2]:
scores = pd.read_csv(os.path.join('data','scores.csv'))
conditions = glob.glob(os.path.join('data','condition','')+'*')
control = glob.glob(os.path.join('data','control','')+'*')

In [3]:
conditions_list = []
control_list = []
for file in conditions:
    d = pd.read_csv(file).describe().T
    d['number']=os.path.basename(file)[:-4]
    conditions_list.append(d)
for file in control:
    d = pd.read_csv(file).describe().T
    d['number']=os.path.basename(file)[:-4]
    control_list.append(d)

In [4]:
control_df = pd.concat(control_list)
conditions_df = pd.concat(conditions_list)

In [5]:
final_data = pd.concat([conditions_df,control_df])
df = scores.merge(final_data,on='number')
df = df.fillna(0)

In [7]:
df = df.drop(['edu','age'],axis=1)

In [9]:

X = df.drop(['afftype','number'],axis=1)
y = df['afftype']
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [14]:
model = lbm.LGBMClassifier()
model.fit(X_train,y_train)

LGBMClassifier()

In [17]:
model = lbm.LGBMClassifier()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')

In [27]:

model.fit(X_train,y_train)
predictions = model.predict(X_test)

In [28]:
X_train

,days,gender,melanch,inpatient,marriage,work,madrs1,madrs2,count,mean,std,min,25%,50%,75%,max
49,13,2,0.0,0.0,0.0,0.0,0.0,0.0,20490.0,314.047194,431.451563,0.0,0.0,130.0,469.0,3869.0
48,13,1,0.0,0.0,0.0,0.0,0.0,0.0,33365.0,231.813128,418.731490,0.0,0.0,0.0,346.0,8000.0
26,13,1,0.0,0.0,0.0,0.0,0.0,0.0,31455.0,138.161055,331.341977,0.0,0.0,3.0,103.0,4354.0
37,11,1,0.0,0.0,0.0,0.0,0.0,0.0,22263.0,278.202039,422.309352,0.0,0.0,71.0,410.0,4935.0
18,13,2,2.0,1.0,2.0,2.0,26.0,21.0,21231.0,157.240215,286.974106,0.0,0.0,19.0,189.0,8000.0
24,20,1,0.0,0.0,0.0,0.0,0.0,0.0,31473.0,398.883456,533.953182,0.0,0.0,141.0,662.0,4927.0
12,14,2,2.0,2.0,2.0,2.0,18.0,13.0,25910.0,221.348746,347.336033,0.0,0.0,36.0,332.0,4859.0
46,13,2,0.0,0.0,0.0,0.0,0.0,0.0,21824.0,351.914589,624.376553,0.0,0.0,102.0,439.0,8000.0
54,14,2,0.0,0.0,0.0,0.0,0.0,0.0,51619.0,68.251613,210.449355,0.0,0.0,0.0,0.0,4009.0
8,13,2,0.0,2.0,1.0,2.0,26.0,26.0,20318.0,178.773304,308.086489,0.0,0.0,32.0,225.0,2821.0
